## 1. Imports


In [ ]:
import os
import sys
import gc
import json
import warnings
from typing import List

import torch
import torch.nn as nn
import numpy as np
from diffusers import DDIMScheduler
from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt
from PIL import PngImagePlugin
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

sys.path.append("..")
from src.u2net import U2NET
from src.unet import UNet
from src.resnet2 import ResNet_D
from src.losses import VGGPerceptualLoss as VGGLoss
from src.tools import (
    set_random_seed,
    unfreeze,
    freeze,
    weights_init_D,
    fig2tensor,
    get_all_pivotal,
    get_step_t_pivotal,
    linked_push,
)  # for wandb
from src.plotters import (
    plot_linked_pushed_images,
    plot_linked_pushed_random_paired_images,
)

from src.samplers import PairedLoaderSampler, get_paired_sampler

LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

warnings.filterwarnings("ignore")

%matplotlib inline 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

## 2. Config

Dataset choosing in the first rows


In [ ]:
SEED = 0x3060
set_random_seed(SEED)

# dataset choosing
# face2comic
DATASET, DATASET_PATH, REVERSE = (
    "comic_faces_v1",
    "../datasets/face2comics_v1.0.0_by_Sxela",
    False,
)

# colored mask -> face
# DATASET, DATASET_PATH, REVERSE = (
#     "celeba_mask",
#     "../datasets/CelebAMask-HQ",
#     False,
# )

# sketch -> face
# DATASET, DATASET_PATH, REVERSE = (
#     "FS2K",
#     "../datasets/FS2K/",
#     False,
# )

IMG_SIZE = 256
DATASET1_CHANNELS = 3
DATASET2_CHANNELS = 3

# the step number adding noise in diffusion process
DIFFUSION_STEPS = 1000
PIVOTAL_LIST = [40, 60, 90]

# GPU choosing
DEVICE_ID = 0
assert torch.cuda.is_available()
torch.cuda.set_device(f"cuda:{DEVICE_ID}")

CONTINUE = [0, 0]  # first is for step, last is for sdes

# All hyperparameters below is set to the values used for the experiments, which discribed in the article

# training algorithm settings
STRATEGY = "Fix"  # 'Fix' or 'Adapt'
T_ITERS = 10
MAX_STEPS = 10000 + 1  # 2501 for testing

COST = "vgg"  #'mse' # 'mae' # 'vgg'
# data sample settings
BATCH_SIZE = 8

# optimizer settings
D_LR, T_LR = 1e-4, 1e-4
T_GRADIENT_MAX_NORM = float(500)
D_GRADIENT_MAX_NORM = float(500)
SCHEDULER_MILESTONES = [2500, 4000]

# network settings
NOT = True  # Train Neural optimal transport or pure regression
T_TYPE = "U2Net"  # 'UNet' # or  ('ResNet_pix2pix' - not implemented)
UNET_BASE_FACTOR = 48  # For UNet
D_TYPE = (
    "ResNet"  # or 'ResNet_pix2pix' - DOES NOT WORK WELL (it is actually not a resnet:)
)
D_USE_BATCHNORM = False  # For ResNet_D

# plot settings
GRAY_PLOTS = False
PLOT_N_SAMPLES = 8

# log settings
TRACK_VAR_INTERVAL = 500
PLOT_INTERVAL = 500
CKPT_INTERVAL = 1000

FID_EPOCHS = 1

EXP_NAME = f"DNOT_Paired_{DATASET}_{STRATEGY}_{SEED}"
OUTPUT_PATH = f"../saved_models/{EXP_NAME}/"

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

writer = SummaryWriter(f"../logdir/{EXP_NAME}")

In [ ]:
config = dict(
    SEED=SEED,
    DATASET=DATASET,
    REVERSE=REVERSE,
    IMG_SIZE=IMG_SIZE,
    DATASET1_CHANNELS=DATASET1_CHANNELS,
    DATASET2_CHANNELS=DATASET2_CHANNELS,
    DIFFUSION_STEPS=DIFFUSION_STEPS,
    PIVOTAL_LIST=PIVOTAL_LIST,
    STRATEGY=STRATEGY,
    BATCH_SIZE=BATCH_SIZE,
    T_ITERS=T_ITERS,
    MAX_STEPS=MAX_STEPS,
    COST=COST,
    NOT=NOT,
    T_TYPE=T_TYPE,
    UNET_BASE_FACTOR=UNET_BASE_FACTOR,
    D_TYPE=D_TYPE,
    D_USE_BATCHNORM=D_USE_BATCHNORM,
    D_LR=D_LR,
    T_LR=T_LR,
    T_GRADIENT_MAX_NORM=T_GRADIENT_MAX_NORM,
    D_GRADIENT_MAX_NORM=D_GRADIENT_MAX_NORM,
    GRAY_PLOTS=GRAY_PLOTS,
    PLOT_N_SAMPLES=PLOT_N_SAMPLES,
    TRACK_VAR_INTERVAL=TRACK_VAR_INTERVAL,
    PLOT_INTERVAL=PLOT_INTERVAL,
    CKPT_INTERVAL=CKPT_INTERVAL,
    FID_EPOCHS=FID_EPOCHS,
)
with open(os.path.join(OUTPUT_PATH, "config.json"), "w") as json_file:
    json_str = json.dumps(config, indent=4)
    json_file.write(json_str)

log = dict(CONTINUE=CONTINUE)
with open(os.path.join(OUTPUT_PATH, "log.json"), "w") as log_file:
    log_str = json.dumps(log, indent=4)
    log_file.write(log_str)

In [ ]:
if COST == "vgg":
    vgg_loss = VGGLoss().cuda()

## 3. Initialize samplers


### data sampler

In [ ]:
XY_sampler, XY_test_sampler = get_paired_sampler(
    DATASET, DATASET_PATH, img_size=IMG_SIZE, batch_size=BATCH_SIZE, reverse=REVERSE
)

In [ ]:
torch.cuda.empty_cache()
gc.collect()
clear_output()

### pivotal sampler


In [ ]:
SCHEDULER = DDIMScheduler(num_train_timesteps=DIFFUSION_STEPS)


def sample_all_pivotal(
    XY_sampler: PairedLoaderSampler,
    batch_size: int = 4,
) -> list[torch.Tensor]:
    source, target = XY_sampler.sample(batch_size)

    return get_all_pivotal(
        source,
        target,
        SCHEDULER,
        PIVOTAL_LIST,
    )

In [ ]:
def plot_all_pivotal(
    source: torch.Tensor,
    target: torch.Tensor,
    gray: bool = False,
) -> list:
    pivotal_path = get_all_pivotal(
        source,
        target,
        SCHEDULER,
        PIVOTAL_LIST,
    )

    imgs: np.ndarray = (
        torch.stack(pivotal_path)
        .to("cpu")
        .permute(0, 2, 3, 1)
        .mul(0.5)
        .add(0.5)
        .numpy()
        .clip(0, 1)
    )
    nrows, ncols = 1, len(pivotal_path)
    fig = plt.figure(figsize=(1.5 * ncols, 1.5 * nrows), dpi=150)
    for i, img in enumerate(imgs):
        ax = fig.add_subplot(nrows, ncols, i + 1)
        if gray:
            ax.imshow(img, cmap="gray")
        else:
            ax.imshow(img)
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(False)
        ax.set_yticks([])
        ax.set_xticks([])
        ax.set_title(f"$X_{i}$", fontsize=24)
        if i == imgs.shape[0] - 1:
            ax.set_title("Y", fontsize=24)

    torch.cuda.empty_cache()
    gc.collect()

## 4. Initialize models


### init models

In [ ]:
Ts, Ds = [], []
T_OPTs, D_OPTs = [], []
T_SCHEDULERs, D_SCHEDULERs = [], []

for i in range(len(PIVOTAL_LIST) * 2):
    if T_TYPE == "UNet":
        T = UNet(
            DATASET1_CHANNELS, DATASET2_CHANNELS, base_factor=UNET_BASE_FACTOR
        ).cuda()
    elif T_TYPE == "U2Net":
        T = U2NET(in_ch=DATASET1_CHANNELS, out_ch=DATASET2_CHANNELS).cuda()
    else:
        raise NotImplementedError(f"Unknown T_TYPE: {T_TYPE}")
    Ts.append(T)

    if D_TYPE == "ResNet":
        D = ResNet_D(
            IMG_SIZE,
            nc=DATASET2_CHANNELS,
            bn=D_USE_BATCHNORM,
        ).cuda()
        D.apply(weights_init_D)
    else:
        raise NotImplementedError(f"Unknown D_TYPE: {D_TYPE}")
    D.apply(weights_init_D)
    Ds.append(D)

    T_opt = torch.optim.Adam(T.parameters(), lr=T_LR, weight_decay=1e-10)
    D_opt = torch.optim.Adam(D.parameters(), lr=D_LR, weight_decay=1e-10)
    T_OPTs.append(T_opt)
    D_OPTs.append(D_opt)

    T_scheduler = torch.optim.lr_scheduler.MultiStepLR(
        T_opt, milestones=SCHEDULER_MILESTONES, gamma=0.5
    )
    D_scheduler = torch.optim.lr_scheduler.MultiStepLR(
        D_opt, milestones=SCHEDULER_MILESTONES, gamma=0.5
    )
    T_SCHEDULERs.append(T_scheduler)
    D_SCHEDULERs.append(D_scheduler)

### load weights

In [ ]:
if CONTINUE[0] > 0 or CONTINUE[1] > 0:
    print("Loading weights for continue training")
    if STRATEGY == "Adapt":
        for i, (T, T_opt, T_scheduler, D, D_opt, D_scheduler) in enumerate(
            zip(
                Ts,
                T_OPTs,
                T_SCHEDULERs,
                Ds,
                D_OPTs,
                D_SCHEDULERs,
            )
        ):
            if i > CONTINUE[1]:
                continue
            if i < CONTINUE[1]:
                CKPT_DIR = os.path.join(OUTPUT_PATH, f"iter{MAX_STEPS - 1}/")
            if i == CONTINUE[1]:
                CKPT_DIR = os.path.join(OUTPUT_PATH, f"iter{CONTINUE[0] - 1}/")

            T.load_state_dict(torch.load(os.path.join(CKPT_DIR, f"T{i}_{SEED}.pt")))
            print(f"{CKPT_DIR} T{i}_{SEED}.pt, loaded")

            T_opt.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"T_opt{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} T_opt{i}_{SEED}.pt, loaded")

            T_scheduler.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"T_scheduler{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} T_scheduler{i}_{SEED}.pt, loaded")

            D.load_state_dict(torch.load(os.path.join(CKPT_DIR, f"D{i}_{SEED}.pt")))
            print(f"{CKPT_DIR} D{i}_{SEED}.pt, loaded")

            D_opt.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"D_opt{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} D_opt{i}_{SEED}.pt, loaded")

            D_scheduler.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"D_scheduler{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} D_scheduler{i}_{SEED}.pt, loaded")

    if STRATEGY == "Fix":
        CKPT_DIR = os.path.join(OUTPUT_PATH, f"iter{CONTINUE[0] - 1}/")
        for i, (T, T_opt, T_scheduler, D, D_opt, D_scheduler) in enumerate(
            zip(
                Ts,
                T_OPTs,
                T_SCHEDULERs,
                Ds,
                D_OPTs,
                D_SCHEDULERs,
            )
        ):
            T.load_state_dict(torch.load(os.path.join(CKPT_DIR, f"T{i}_{SEED}.pt")))
            print(f"{CKPT_DIR} T{i}_{SEED}.pt, loaded")

            T_opt.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"T_opt{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} T_opt{i}_{SEED}.pt, loaded")

            T_scheduler.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"T_scheduler{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} T_scheduler{i}_{SEED}.pt, loaded")

            D.load_state_dict(torch.load(os.path.join(CKPT_DIR, f"D{i}_{SEED}.pt")))
            print(f"{CKPT_DIR} D{i}_{SEED}.pt, loaded")

            D_opt.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"D_opt{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} D_opt{i}_{SEED}.pt, loaded")

            D_scheduler.load_state_dict(
                torch.load(os.path.join(CKPT_DIR, f"D_scheduler{i}_{SEED}.pt"))
            )
            print(f"{CKPT_DIR} D_scheduler{i}_{SEED}.pt, loaded")

## 5. Plots Test


In [ ]:
X_test_fixed, Y_test_fixed = XY_test_sampler.sample(PLOT_N_SAMPLES)

In [ ]:
fig, axes = plot_all_pivotal(
    X_test_fixed[0],
    Y_test_fixed[0],
    GRAY_PLOTS,
)

In [ ]:
fig, axes = plot_linked_pushed_images(
    X_test_fixed,
    Y_test_fixed,
    Ts,
    gray=GRAY_PLOTS,
)

In [ ]:
fig, axes = plot_linked_pushed_random_paired_images(
    XY_sampler,
    Ts,
    plot_n_samples=PLOT_N_SAMPLES,
    gray=GRAY_PLOTS,
)

## 6. Train


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def get_cat_pivotal_tr(XY_sampler1, XY_sampler2, batch_size):
    X_tr = sample_all_pivotal(XY_sampler1, batch_size)
    Y_tr = sample_all_pivotal(XY_sampler2, batch_size)
    assert len(X_tr) == len(Y_tr)

    tr = []
    length = len(X_tr)
    mid_indx = length // 2
    tr.extend(X_tr[: mid_indx + 1])
    tr.extend(Y_tr[mid_indx + 1 :])

    del X_tr[mid_indx + 1 :]
    del Y_tr[: mid_indx + 1]

    gc.collect()
    torch.cuda.empty_cache()
    return tr

### Fix strategy


In [ ]:
if STRATEGY == "Fix":
    pbar = tqdm(total=MAX_STEPS, initial=max(0, CONTINUE[0]), leave=True)

#### training

In [ ]:
if STRATEGY == "Fix":
    for step in range(max(0, CONTINUE[0]), MAX_STEPS):
        for k in range(T_ITERS):
            # === sample path ===
            pivotal_path = sample_all_pivotal(XY_sampler, BATCH_SIZE)
            for i, (T, T_opt, D) in enumerate(zip(Ts, T_OPTs, Ds)):
                freeze(D)
                unfreeze(T)
                # === sample input ===
                X, Y = pivotal_path[i], pivotal_path[i + 1]
                # === clear grad ===
                T_opt.zero_grad()
                # === forward ===
                T_X = T(X)
                # === loss ===
                if COST == "rmse":
                    T_loss = (Y - T_X[:, :3]).flatten(start_dim=1).norm(dim=1).mean()
                elif COST == "mse":
                    T_loss = (
                        (Y - T_X[:, :3]).flatten(start_dim=1).square().sum(dim=1).mean()
                    )
                elif COST == "mae":
                    T_loss = (
                        (Y - T_X[:, :3]).flatten(start_dim=1).abs().sum(dim=1).mean()
                    )
                elif COST == "vgg":
                    T_loss = vgg_loss(Y, T_X[:, :3]).mean()
                else:
                    raise Exception("Unknown COST")

                if NOT:
                    T_loss -= D(T_X).mean()

                writer.add_scalar(f"T_loss/T{i}", T_loss.item(), step)
                # === backward ===
                T_loss.backward()
                # === clip grad ===
                T_gradient_norm = torch.nn.utils.clip_grad_norm_(
                    T.parameters(), max_norm=T_GRADIENT_MAX_NORM
                )
                # === optim ===
                T_opt.step()
            #     # === clear tmp variables ===
            #     del X, Y, T_X, T_loss
            #     gc.collect()
            #     torch.cuda.empty_cache()
            # del pivotal_path
            # gc.collect()
            # torch.cuda.empty_cache()
        # === update lr ===
        for T_scheduler in T_SCHEDULERs:
            T_scheduler.step()

        if NOT:
            for T, D in zip(Ts, Ds):
                freeze(T)
                freeze(D)
            # === sample path ===
            # two way
            #   1. concatnat path: pivotal_path = get_cat_pivotal_tr(XY_sampler, XY_sampler, BATCH_SIZE)
            #   2. just 1 path: pivotal_path = sample_all_pivotal(XY_sampler, BATCH_SIZE)
            pivotal_path = sample_all_pivotal(XY_sampler, BATCH_SIZE)
            for i, (D, D_opt, D_scheduler, T) in enumerate(
                zip(Ds, D_OPTs, D_SCHEDULERs, Ts)
            ):
                freeze(T)
                unfreeze(D)
                # === sample input ===
                X, Y = pivotal_path[i], pivotal_path[i + 1]
                # === clear grad ===
                D_opt.zero_grad()
                # === forward ===
                with torch.no_grad():
                    T_X = T(X)
                # === loss ===
                D_loss = D(T_X).mean() - D(Y).mean()
                writer.add_scalar(f"D_loss/D{i}", D_loss.item(), step)
                # === backward ===
                D_loss.backward()
                # === clip grad ===
                D_gradient_norm = torch.nn.utils.clip_grad_norm_(
                    D.parameters(), max_norm=D_GRADIENT_MAX_NORM
                )
                # === optim ===
                D_opt.step()
                # === update lr ===
                D_scheduler.step()
            #     # === clear tmp variables ===
            #     del X, Y, T_X, D_loss
            #     torch.cuda.empty_cache()
            #     gc.collect()
            # del pivotal_path
            # gc.collect()
            # torch.cuda.empty_cache()

        CONTINUE[0] += 1
        pbar.update(1)

        if step % PLOT_INTERVAL == 0:
            clear_output(wait=True)
            print(f"{step = }, Plotting")

            print("Fixed Test Images")
            fig, axes = plot_linked_pushed_images(
                X_test_fixed,
                Y_test_fixed,
                Ts,
                gray=GRAY_PLOTS,
            )
            writer.add_image("Fixed Test Images", fig2tensor(fig), step)
            plt.show(fig)
            plt.close(fig)

            print("Random Test Images")
            fig, axes = plot_linked_pushed_random_paired_images(
                XY_test_sampler,
                Ts,
                plot_n_samples=PLOT_N_SAMPLES,
                gray=GRAY_PLOTS,
            )
            writer.add_image("Random Test Images", fig2tensor(fig), step)
            plt.show(fig)
            plt.close(fig)

        if step != 0 and step % CKPT_INTERVAL == 0:
            CKPT_DIR = os.path.join(OUTPUT_PATH, f"iter{step}/")
            os.makedirs(CKPT_DIR, exist_ok=True)
            for i, (T, T_opt, T_scheduler, D, D_opt, D_scheduler) in enumerate(
                zip(
                    Ts,
                    T_OPTs,
                    T_SCHEDULERs,
                    Ds,
                    D_OPTs,
                    D_SCHEDULERs,
                )
            ):
                torch.save(T.state_dict(), os.path.join(CKPT_DIR, f"T{i}_{SEED}.pt"))
                torch.save(D.state_dict(), os.path.join(CKPT_DIR, f"D{i}_{SEED}.pt"))

                torch.save(
                    D_opt.state_dict(),
                    os.path.join(CKPT_DIR, f"D_opt{i}_{SEED}.pt"),
                )
                torch.save(
                    T_opt.state_dict(),
                    os.path.join(CKPT_DIR, f"T_opt{i}_{SEED}.pt"),
                )
                torch.save(
                    D_scheduler.state_dict(),
                    os.path.join(CKPT_DIR, f"D_scheduler{i}_{SEED}.pt"),
                )
                torch.save(
                    T_scheduler.state_dict(),
                    os.path.join(CKPT_DIR, f"T_scheduler{i}_{SEED}.pt"),
                )

            log["CONTINUE"] = CONTINUE
            with open(os.path.join(OUTPUT_PATH, "log.json"), "w") as log_file:
                log_str = json.dumps(log, indent=4)
                log_file.write(log_str)

        if step % TRACK_VAR_INTERVAL == 0:
            pass
            # print("Computing FID")
            # mu, sigma = get_linked_sde_pushed_loader_stats(
            #     Ts,
            #     XY_test_sampler.loader,
            #     n_epochs=FID_EPOCHS,
            #     batch_size=BATCH_SIZE,
            #     verbose=True,
            # )
            # fid = calculate_frechet_distance(mu_data, sigma_data, mu, sigma)
            # print(f"FID={fid}")
            # writer.add_scalar("Metrics/FID", fid, step)
            # del mu, sigma

            # print("Computing LPIPS(vgg) LPIPS(alex) L1 MSE")
            # metrics = get_linked_sde_pushed_loader_metrics(
            #     Ts,
            #     XY_test_sampler.loader,
            #     n_epochs=FID_EPOCHS,
            #     batch_size=BATCH_SIZE,
            #     verbose=True,
            #     log_metrics=["mse", "l1"]
            # )
            # print(f"metrics={metrics}")
            # writer.add_scalar("Metrics/LPIPS(VGG)", metrics["vgg"], step)
            # writer.add_scalar("Metrics/LPIPS(Alex)", metrics["alex"], step)
            # writer.add_scalar("Metrics/L1", metrics["l1"], step)
            # writer.add_scalar("Metrics/MSE", metrics["mse"], step)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

### Adapt strategy


In [ ]:
if STRATEGY == "Adapt":
    spbar = tqdm(total=len(Ts), initial=0, position=0, desc="total")
    pbar = tqdm(
        total=MAX_STEPS,
        initial=max(0, CONTINUE[0]),
        position=1,
        desc="single",
        leave=True,
    )

#### training

In [ ]:
if STRATEGY == "Adapt":
    for i, (T, T_opt, T_scheduler, D, D_opt, D_scheduler) in enumerate(
        zip(
            Ts,
            T_OPTs,
            T_SCHEDULERs,
            Ds,
            D_OPTs,
            D_SCHEDULERs,
        )
    ):
        if i < CONTINUE[1]:
            spbar.update(1)
            continue

        for _T, _D in zip(Ts, Ds):
            freeze(_T)
            freeze(_D)

        for step in range(max(0, CONTINUE[0]), MAX_STEPS):
            for k in range(T_ITERS):
                freeze(D)
                unfreeze(T)
                # === sample path ===
                pivotal_path = sample_all_pivotal(XY_sampler, BATCH_SIZE)
                # === sample input ===
                X, Y = pivotal_path[0], pivotal_path[i + 1]
                X = linked_push(Ts[:i], X, "T_X")
                # X.requires_grad_(True)
                # Y.requires_grad_(False)

                # === clear grad ===
                T_opt.zero_grad()
                # === forward ===
                T_X = T(X)
                # === loss ===
                if COST == "rmse":
                    T_loss = (Y - T_X[:, :3]).flatten(start_dim=1).norm(dim=1).mean()
                elif COST == "mse":
                    T_loss = (
                        (Y - T_X[:, :3]).flatten(start_dim=1).square().sum(dim=1).mean()
                    )
                elif COST == "mae":
                    T_loss = (
                        (Y - T_X[:, :3]).flatten(start_dim=1).abs().sum(dim=1).mean()
                    )
                elif COST == "vgg":
                    T_loss = vgg_loss(Y, T_X[:, :3]).mean()
                else:
                    raise Exception("Unknown COST")

                if NOT:
                    T_loss -= D(T_X).mean()

                writer.add_scalar(f"T_loss/T{i}", T_loss.item(), step)
                # === backward ===
                T_loss.backward()
                # === clip grad ===
                T_gradient_norm = torch.nn.utils.clip_grad_norm_(
                    T.parameters(), max_norm=T_GRADIENT_MAX_NORM
                )
                # === optim ===
                T_opt.step()
                # # === clear tmp variables ===
                # del pivotal_path, X, Y, T_X, T_loss
                # gc.collect()
                # torch.cuda.empty_cache()
            # === update lr ===
            T_scheduler.step()

            if NOT:
                freeze(T)
                unfreeze(D)
                # === sampler path ===
                # two way
                #   1. concatnat path: pivotal_path = get_cat_pivotal_tr(XY_sampler, XY_sampler, BATCH_SIZE)
                #   2. just 1 path: pivotal_path = sample_all_pivotal(XY_sampler, BATCH_SIZE)
                pivotal_path = sample_all_pivotal(XY_sampler, BATCH_SIZE)
                # === sample input ===
                X, Y = pivotal_path[0], pivotal_path[i + 1]
                X = linked_push(Ts[:i], X, "T_X")
                # X0.requires_grad_(True)
                # X1.requires_grad_(False)

                # === clear grad ===
                D_opt.zero_grad()
                # === forward ===
                with torch.no_grad():
                    T_X = T(X)
                # === loss ===
                D_loss = D(T_X).mean() - D(Y).mean()
                writer.add_scalar(f"D_loss/D{i}", D_loss.item(), step)
                # === backward ===
                D_loss.backward()
                # === clip grad ===
                D_gradient_norm = torch.nn.utils.clip_grad_norm_(
                    D.parameters(), max_norm=D_GRADIENT_MAX_NORM
                )
                # === optim ===
                D_opt.step()
                # === update lr ===
                D_scheduler.step()
                # # === clear tmp variables ===
                # del pivotal_path, X, Y, T_X, D_loss
                # gc.collect()
                # torch.cuda.empty_cache()
            CONTINUE[0] += 1
            pbar.update(1)

            if step % PLOT_INTERVAL == 0:
                clear_output(wait=True)
                print(f"training {i}-th T, {step = }, Plotting")

                print("Fixed Test Images")
                fig, axes = plot_linked_pushed_images(
                    X_test_fixed,
                    Y_test_fixed,
                    Ts,
                    gray=GRAY_PLOTS,
                )
                writer.add_image(
                    f"Fix Test Images/T[0..={i}]",
                    fig2tensor(fig),
                    step,
                )
                plt.show(fig)
                plt.close(fig)

                print("Random Test Images")
                fig, axes = plot_linked_pushed_random_paired_images(
                    XY_test_sampler,
                    Ts,
                    plot_n_samples=PLOT_N_SAMPLES,
                    gray=GRAY_PLOTS,
                )
                writer.add_image(
                    f"Random Test Images/T[0..={i}]",
                    fig2tensor(fig),
                    step,
                )
                plt.show(fig)
                plt.close(fig)

            if step != 0 and step % CKPT_INTERVAL == 0:
                CKPT_DIR = os.path.join(OUTPUT_PATH, f"iter{step}/")
                os.makedirs(CKPT_DIR, exist_ok=True)
                torch.save(T.state_dict(), os.path.join(CKPT_DIR, f"T{i}_{SEED}.pt"))
                torch.save(D.state_dict(), os.path.join(CKPT_DIR, f"D{i}_{SEED}.pt"))

                torch.save(
                    D_opt.state_dict(),
                    os.path.join(CKPT_DIR, f"D_opt{i}_{SEED}.pt"),
                )
                torch.save(
                    T_opt.state_dict(),
                    os.path.join(CKPT_DIR, f"T_opt{i}_{SEED}.pt"),
                )
                torch.save(
                    D_scheduler.state_dict(),
                    os.path.join(CKPT_DIR, f"D_scheduler{i}_{SEED}.pt"),
                )
                torch.save(
                    T_scheduler.state_dict(),
                    os.path.join(CKPT_DIR, f"T_scheduler{i}_{SEED}.pt"),
                )
                log = dict(CONTINUE=CONTINUE)
                with open(os.path.join(OUTPUT_PATH, "log.json"), "w") as log_file:
                    log_str = json.dumps(log, indent=4)
                    log_file.write(log_str)

            if i == len(Ts) - 1 and step % TRACK_VAR_INTERVAL == 0:
                pass
                # print("Computing FID")
                # mu, sigma = get_linked_sde_pushed_loader_stats(
                #     Ts,
                #     XY_test_sampler.loader,
                #     n_epochs=FID_EPOCHS,
                #     batch_size=BATCH_SIZE,
                #     verbose=True,
                # )
                # fid = calculate_frechet_distance(mu_data, sigma_data, mu, sigma)
                # writer.add_scalar(f"Metrics/FID{i}", fid, step)
                # del mu, sigma

                # print("Computing LPIPS(vgg) LPIPS(alex) L1 MSE")
                # metrics = get_linked_sde_pushed_loader_metrics(
                #     Ts,
                #     XY_test_sampler.loader,
                #     n_epochs=FID_EPOCHS,
                #     batch_size=BATCH_SIZE,
                #     verbose=True,
                #     log_metrics=["mse", "l1"]
                # )
                # print(f"metrics={metrics}")
                # writer.add_scalar("LPIPS(VGG)", metrics["vgg"], step)
                # writer.add_scalar("LPIPS(Alex)", metrics["alex"], step)
                # writer.add_scalar("L1", metrics["l1"], step)
                # writer.add_scalar("MSE", metrics["mse"], step)

            gc.collect()
            torch.cuda.empty_cache()

        CONTINUE[0] = 0  # reset training steps to 0
        pbar.reset()
        CONTINUE[1] += 1
        spbar.update(1)

## Clear resources


In [ ]:
try:
    writer.close()
    pbar.close()
    spbar.close()
except Exception as e:
    print(e)